# 공공데이터 포털 동네예보 API를 활용해 내일 기상 예보 받아보기

코드 출처: https://dacon.io/competitions/official/235720/codeshare/2555?page=2&dtype=recent

공공데이터 포털 동네예보 API 바로가기 => https://data.go.kr/data/15057682/openapi.do

API는 신청하여 인증키를 발급 받은 후 사용 가능합니다.

자세한 사용법은 해당 페이지 '기상청18_동네예보 조회서비스_오픈API활용가이드.zip'를 통해 확인 가능합니다.

사용방법 https://andjjip.tistory.com/108 참조

In [10]:
import pandas as pd
import urllib
import urllib.request
import json


API 사용 신청후 발급받은 일반 인증키(Decoding)를 ServiceKey.txt에 저장하여 불러왔습니다.

인증키 확인 페이지 바로가기(신청 후 사용 가능) => https://data.go.kr/iim/api/selectAPIAcountView.do

In [2]:
ServiceKey = open('ServiceKey.txt', 'r').read()

In [25]:
url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService/getVilageFcst' #동네예보조회

queryParams = '?' + urllib.parse.urlencode(
    {
        urllib.parse.quote_plus('ServiceKey') : 'nsy9UUyARVZjVQzNJjmcW%2BwojTpc6ounUkI%2FJZS5M8hvocfTa9PZBsu7MiqGUFDVJn2mo4dLHnoxXL8Vhhuzcg%3D%3D', # key를 바로 입력해도 됩니다.
        urllib.parse.quote_plus('numOfRows') : '113', # 총 14개의 항목을 3시간 단위로 순차적으로 불러옵니다. 다음날 24시간예보에 필요한 만큼만 가져왔습니다.
        urllib.parse.quote_plus('dataType') : 'JSON', # JSON, XML 두가지 포멧을 제공합니다.
        urllib.parse.quote_plus('base_date') : '20210611', # 예보 받을 날짜를 입력합니다. 최근 1일간의 자료만 제공합니다.
        urllib.parse.quote_plus('base_time') : '1400', # 예보 시간을 입력합니다. 2시부터 시작하여 3시간 단위로 입력 가능합니다.
        urllib.parse.quote_plus('nx') : '102', # 울산 태양광 발전소 x 좌표입니다. '기상청18_동네예보 조회서비스_오픈API활용가이드.zip'에 포함 된 excel파일을 통해 확인 가능합니다.
        urllib.parse.quote_plus('ny') : '83' # 울산 태양광 발전소 y 좌표입니다. '기상청18_동네예보 조회서비스_오픈API활용가이드.zip'에 포함 된 excel파일을 통해 확인 가능합니다.
    }
)

response = urllib.request.urlopen(url + queryParams).read()
response = json.loads(response)

In [26]:
response

b'<OpenAPI_ServiceResponse>\n\t<cmmMsgHeader>\n\t\t<errMsg>SERVICE ERROR</errMsg>\n\t\t<returnAuthMsg>SERVICE_KEY_IS_NOT_REGISTERED_ERROR</returnAuthMsg>\n\t\t<returnReasonCode>30</returnReasonCode>\n\t</cmmMsgHeader>\n</OpenAPI_ServiceResponse>'

In [ ]:
fcst_df = pd.DataFrame()
date = '2021-06-11'
fcst_df['Forecast_time'] = [f'{date} {hour}:00' for hour in range(24)]
row_idx = 0

for i, data in enumerate(response['response']['body']['items']['item']):
    if i > 19:
        if data['category']=='REH':
            fcst_df.loc[row_idx, 'Humidity'] = float(data['fcstValue'])
            print('category:Humidity,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])
        elif data['category']=='T3H':
            fcst_df.loc[row_idx, 'Temperature'] = float(data['fcstValue'])
            print('category:Temperature,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])
        elif data['category']=='SKY':
            fcst_df.loc[row_idx, 'Cloud'] = float(data['fcstValue'])
            print('category:Cloud,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])
        elif data['category']=='VEC':
            fcst_df.loc[row_idx, 'WindDirection'] = float(data['fcstValue'])
            print('category:WindDirection,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])
        elif data['category']=='WSD':
            fcst_df.loc[row_idx, 'WindSpeed'] = float(data['fcstValue'])
            print('category:WindSpeed,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'], '\n')
            row_idx+=3

예보를 한시간 단위로 선형보간합니다.

In [5]:
fcst_df = fcst_df.interpolate()
fcst_df = fcst_df.iloc[:24]

In [6]:
fcst_df

,Forecast_time,Humidity,Cloud,Temperature,WindDirection,WindSpeed
0,2021-04-21 0:00,75.000000,3.000000,13.000000,346.000000,0.800000
1,2021-04-21 1:00,76.666667,3.000000,12.666667,332.000000,0.900000
2,2021-04-21 2:00,78.333333,3.000000,12.333333,318.000000,1.000000
3,2021-04-21 3:00,80.000000,3.000000,12.000000,304.000000,1.100000
4,2021-04-21 4:00,78.333333,3.000000,11.666667,300.333333,1.166667
5,2021-04-21 5:00,76.666667,3.000000,11.333333,296.666667,1.233333
6,2021-04-21 6:00,75.000000,3.000000,11.000000,293.000000,1.300000
7,2021-04-21 7:00,68.333333,3.000000,13.333333,202.666667,1.466667
8,2021-04-21 8:00,61.666667,3.000000,15.666667,112.333333,1.633333
9,2021-04-21 9:00,55.000000,3.000000,18.000000,22.000000,1.800000
